In [ ]:
#Importing the necessary data:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time
import pandas as pd
import re
import numpy as np
from datetime import datetime, timedelta
import datetime
import requests
import json
import random

# **Inventory**

In [ ]:
#Reading in our Books data-frame:
books_info = pd.read_csv('/content/book_info.csv', index_col = 0)

#Observing our books data-frame:
display(books_info)

,Title,Author(s),Rating,Rating_count,Review_count,No. of Pages,Format,Awards,Genres
1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling,4.58,3540997.0,66336.0,652.0,Paperback,"Locus Award Best Young Adult Novel (2006), Aud...","Fantasy, Fiction, Young Adult, Magic, Audioboo..."
2,Harry Potter and the Order of the Phoenix (Har...,"J.K. Rowling, Mary GrandPré",4.50,3666223.0,73250.0,912.0,Paperback,Bram Stoker Award Works for Young Readers (200...,"Fantasy, Young Adult, Fiction, Magic, Audioboo..."
3,Harry Potter and the Sorcerer&apos;s Stone (Ha...,J.K. Rowling,4.47,10984901.0,182647.0,309.0,Hardcover,Mythopoeic Fantasy Award Children&apos;s Liter...,"Fantasy, Fiction, Young Adult, Magic, Children..."
4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.43,4316879.0,91245.0,352.0,Hardcover,Mythopoeic Fantasy Award Children&apos;s Liter...,"Fantasy, Fiction, Young Adult, Magic, Children..."
5,Harry Potter and the Prisoner of Azkaban (Harr...,"J.K. Rowling, Mary GrandPré",4.58,4639665.0,96500.0,435.0,Mass Market Paperback,Bram Stoker Award Best Work for Young Readers ...,"Fantasy, Fiction, Young Adult, Magic, Children..."
...,...,...,...,...,...,...,...,...,...
1927,Collapse: How Societies Choose to Fail or Survive,Jared Diamond,3.93,73697.0,3907.0,592.0,Hardcover,"Royal Society Science Book Prize (2006), Calif...","History, Nonfiction, Science, Anthropology, So..."
1928,Colapso: Por qué unas sociedades perduran y ot...,Jared Diamond,3.93,73697.0,3907.0,752.0,Hardcover,"Royal Society Science Book Prize (2006), Calif...","History, Nonfiction, Science, Anthropology, So..."
1929,J. K. Rowling: A Biography (Unauthorized Edition),Connie Ann Kirk,3.46,114.0,12.0,155.0,Hardcover,NaN,"Biography, Nonfiction"
1930,The J.K. Rowling Encyclopedia,Connie Ann Kirk,4.42,24.0,NaN,374.0,Hardcover,NaN,NaN


In [ ]:
#Seeing the number of books:
no_of_books = len(books_info)
print (f'We have {no_of_books} books.')

We have 1931 books.


In [ ]:
#Creating product_ids:
product_id = range(1,no_of_books + 1)

#Creating book_ids:
book_ids = range(1,no_of_books + 1)

#Randomising available quantity:
quantity_available = list()
for _ in range(len(product_id)):
  quantity_available += [random.choice(range(1,1000))]

print ('Length of quantity_available: ',len(quantity_available),'.')

#Randomising prices:
possible_prices = [9.99,14.99,19.99,24.99,30.99]
product_prices = list()
for _ in range(len(product_id)):
  product_prices += [random.choice(possible_prices)]

print ('Length of product_prices: ',len(product_prices),'.')

Length of quantity_available:  1931 .
Length of product_prices:  1931 .


In [ ]:
#Creating our inventory table:
inventory_df = {'BookID': book_ids, 'Price': product_prices, 'Stock' : quantity_available}
inventory_df = pd.DataFrame(inventory_df, index = product_id)

display(inventory_df)

,BookID,Price,Stock
1,1,14.99,516
2,2,9.99,906
3,3,19.99,973
4,4,24.99,186
5,5,24.99,271
...,...,...,...
1927,1927,30.99,688
1928,1928,30.99,299
1929,1929,19.99,512
1930,1930,9.99,579


In [ ]:
inventory_df.to_csv('inventory.csv')

# **Payment Info**

In [ ]:
#Function to generate numbers:
def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return random.randint(range_start, range_end)

In [ ]:
#Importing our users informations:
users_info = pd.read_csv('/content/users_info.csv', index_col = 0)
display(users_info)

,Username,First Name,Last Name,EmailAddress,DOB,AddressID
1,OnniePesach,Onnie,Pesach,Onnie@yahoo.com,2022-06-11,1
2,Heymi.Berenice,Heymi,Berenice,Heymi@yahoo.com,1943-01-29,2
3,Terran_Yolanda,Terran,Yolanda,Terran@gmail.com,2006-09-05,3
4,Awilda.Hikari,Awilda,Hikari,Awilda@hotmail.com,1967-11-27,4
5,Paulette.Vance,Paulette,Vance,Paulette@hotmail.com,1926-09-21,5
...,...,...,...,...,...,...
12641,Arya,Arya,Arya,Arya@gmail.com,1963-01-22,12641
12642,Olivia Cornwell,Olivia,Cornwell,Olivia@gmail.com,2009-09-11,12642
12643,Cheri Herald,Cheri,Herald,Cheri@hotmail.com,2023-04-07,12643
12644,Susy,Susy,Susy,Susy@yahoo.com,1940-01-21,12644


In [ ]:
len_card_no = 16

no_users = len(users_info)

payment_ids = range(1,no_users + 1)
user_ids = range(1,no_users+1)

#Creating card details for all users:
card_type = ['VISA', 'MASTERCARD','American Express','Discover']
card_types = list()
for _ in range(no_users):
  card_types += [random.choice(card_type)]

#Randomising card numbers:
card_numbers = list()
for _ in range(no_users):
  card_numbers += [random_with_N_digits(len_card_no)]

In [ ]:
#Creating our payment method data-frame:
payment_df = {'UserID':user_ids, 'CardNumbers': card_numbers, 'CardType': card_types}
payment_df = pd.DataFrame(payment_df, index = payment_ids)

#Displaying our payment_df:
display(payment_df)

#Saving our data-frame to a csv:
payment_df.to_csv('payment-methods.csv')

,UserID,CardNumbers,CardType
1,1,9357139116450551,MASTERCARD
2,2,9842515504538859,MASTERCARD
3,3,7898669725712000,American Express
4,4,9452517480951574,Discover
5,5,8553703310464875,Discover
...,...,...,...
12641,12641,3084818301254896,VISA
12642,12642,8213870384239320,Discover
12643,12643,2629265914811793,Discover
12644,12644,4502678162616256,Discover
